# [입찰메이트] RAG 시스템 구축 파이프라인

OpenAI (Embeddings, GPT-4o)와 ChromaDB를 사용하여 공공 입찰 공고(RFP) 분석 시스템을 구축합니다.

## 1. 환경 설정 및 라이브러리 설치
HWP 처리를 위한 pyhwp와 PDF 처리를 위한 pymupdf, 그리고 RAG 핵심 라이브러리를 설치합니다.

In [1]:
# 1. HWP 처리를 위한 pyhwp
!pip install -q --pre pyhwp

# 2. PDF 처리(PyMuPDF) 및 RAG/VectorDB 필수 라이브러리
!pip install -q pymupdf langchain langchain-community langchain-openai chromadb pandas tqdm tiktoken

import os
import zipfile
import pandas as pd
import fitz  # PyMuPDF
import subprocess
import re
import getpass
from google.colab import drive
from tqdm import tqdm

# LangChain Imports
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("환경 설정 및 라이브러리 설치 완료")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━

## 2. 데이터 준비 (Drive Mount & Unzip)
구글 드라이브에 있는 원본 데이터(files.zip)와 메타데이터(data_list.csv)를 가져옵니다.

In [3]:
# 1. 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 경로 설정
BASE_PATH = '/content/drive/MyDrive'
ZIP_FILE = os.path.join(BASE_PATH, 'files.zip')
CSV_FILE = os.path.join(BASE_PATH, 'data_list.csv')
EXTRACT_PATH = os.path.join(BASE_PATH, 'files_extracted')

# 3. 압축 해제 (중복 방지)
if not os.path.exists(EXTRACT_PATH):
    os.makedirs(EXTRACT_PATH, exist_ok=True)
    print(f"압축 해제 시작: {ZIP_FILE}")
    try:
        with zipfile.ZipFile(ZIP_FILE, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_PATH)
        print("압축 해제 완료")
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다. 경로 확인 필요: {ZIP_FILE}")
else:
    print(f"압축 해제된 폴더가 이미 존재합니다. (Skip)")

# 4. 실제 파일 경로 보정 (files.zip 안에 files 폴더가 있는지 확인)
FILE_DIR = os.path.join(EXTRACT_PATH, 'files')
if not os.path.exists(FILE_DIR):
    FILE_DIR = EXTRACT_PATH  # 폴더가 없으면 상위 경로 사용

print(f"데이터 작업 경로: {FILE_DIR}")

# 5. 메타데이터 CSV 로드
if os.path.exists(CSV_FILE):
    df = pd.read_csv(CSV_FILE)
    print(f"메타데이터 로드 완료: 총 {len(df)}건")
else:
    print(f"오류: 메타데이터 CSV가 없습니다: {CSV_FILE}")

Mounted at /content/drive
압축 해제된 폴더가 이미 존재합니다. (Skip)
데이터 작업 경로: /content/drive/MyDrive/files_extracted/files
메타데이터 로드 완료: 총 100건


## 3. 텍스트 추출 엔진 구현 (HWP & PDF)
문서의 내용을 깨끗하게 추출하는 것은 RAG 성능의 80%를 결정합니다. 문서 포맷별 최적화된 추출 함수를 정의합니다. hwp5txt와 PyMuPDF를 사용합니다.

- HWP: hwp5txt 명령어를 서브프로세스로 호출하여 텍스트만 추출

- PDF: PyMuPDF를 사용하여 레이아웃 정보 없이 텍스트 스트림 추출

- 공통: clean_text 함수를 통해 불필요한 공백, 탭, 과도한 줄바꿈 등 노이즈 제거

In [4]:
def clean_text(text):
    """텍스트 정제: 불필요한 공백 및 과도한 줄바꿈 제거"""
    if not text: return ""
    text = re.sub(r' +', ' ', text)          # 연속 공백 제거
    text = re.sub(r'\n{3,}', '\n\n', text)   # 3줄 이상 줄바꿈 -> 2줄(문단)로 축소
    return text.strip()

def get_hwp_text(file_path):
    """hwp5txt 명령어를 이용한 HWP 텍스트 추출"""
    try:
        result = subprocess.run(['hwp5txt', file_path], capture_output=True, text=True, encoding='utf-8')
        return clean_text(result.stdout) if result.returncode == 0 else ""
    except Exception:
        return ""

def get_pdf_text(file_path):
    """PyMuPDF를 이용한 PDF 텍스트 추출"""
    try:
        doc = fitz.open(file_path)
        text = "\n".join([page.get_text() for page in doc])
        return clean_text(text)
    except Exception:
        return ""

def load_file_content(file_path):
    """확장자 자동 감지 및 추출"""
    ext = file_path.split('.')[-1].lower()
    if ext == 'hwp':
        return get_hwp_text(file_path)
    elif ext == 'pdf':
        return get_pdf_text(file_path)
    return ""

### 4. [검증] 텍스트 추출 샘플 테스트
전체 데이터를 처리하기 전에, HWP와 PDF가 정상적으로 읽히는지 확인합니다.

In [5]:
import random

def validate_extraction(file_dir):
    print("[사전 검증] 샘플 파일 텍스트 추출 테스트...\n")
    try:
        all_files = os.listdir(file_dir)
    except FileNotFoundError:
        print("오류: 파일 폴더를 찾을 수 없습니다.")
        return

    # HWP, PDF 각 1개씩 랜덤 선택
    samples = [f for f in all_files if f.lower().endswith('.hwp')][:1] + \
              [f for f in all_files if f.lower().endswith('.pdf')][:1]

    if not samples:
        samples = all_files[:2] # 특정 확장자가 없으면 아무거나 2개

    for filename in samples:
        file_path = os.path.join(file_dir, filename)
        content = load_file_content(file_path)

        print(f"파일: {filename}")
        if content:
            print(f"성공 (글자수: {len(content)})")
            print(f"미리보기: {content[:100]} ...\n")
        else:
            print("실패 (내용 없음)\n")

validate_extraction(FILE_DIR)

[사전 검증] 샘플 파일 텍스트 추출 테스트...

파일: 세종테크노파크_세종테크노파크 인사정보 전산시스템 구축 용역 입찰공.hwp
성공 (글자수: 22226)
미리보기: <표>

2021. 9.

<그림>

<표>

<표>

<표>

1. 일반사항
 가. 용 역 명 : 세종테크노파크 인사정보 전산시스템 구축
 나. 용역기간 : 계약 체결일로부터 약 ...

파일: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
성공 (글자수: 45396)
미리보기: 2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
2024. 10.
중이온가속기연구소
가속기운영부 극저온팀

문서번호
-
개정번호
0
발 행 일
2024. 10. 30
 ...



## 5. 데이터 처리 파이프라인 (메타데이터 결합 & 문맥 보강)
문서를 로드하고, 문맥 보강(Context Enrichment) 기술을 적용하여 Document 객체를 생성합니다.

In [6]:
documents = []
print(f"전체 문서 처리 시작 (대상: {len(df)}건)...")

for idx, row in tqdm(df.iterrows(), total=len(df)):
    file_name = row['파일명']
    full_path = os.path.join(FILE_DIR, file_name)

    if not os.path.exists(full_path): continue

    # 텍스트 추출
    content = load_file_content(full_path)
    if not content or len(content) < 50: continue # 내용 너무 짧으면 스킵

    # 메타데이터 매핑 (CSV 컬럼명 호환성 처리)
    metadata = {
        "original_filename": file_name,
        "notice_id": str(row.get('공고 번호', row.get('공고번호', 'N/A'))),
        "agency": str(row.get('발주 기관', row.get('발주기관', 'N/A'))),
        "title": str(row.get('사업명', row.get('공고명', 'N/A')))
    }

    # Context Enrichment: 본문 앞에 메타데이터 강제 주입
    # 문서가 쪼개져도(Chunking) 이 조각이 어떤 사업인지 알 수 있게 함
    enriched_content = f"""[[사업 개요]]
사업명: {metadata['title']}
발주기관: {metadata['agency']}
공고번호: {metadata['notice_id']}

[[본문]]
{content}"""

    # Document 객체 생성
    doc = Document(page_content=enriched_content, metadata=metadata)
    documents.append(doc)

print(f"\n처리 완료: 총 {len(documents)}개의 Document 객체 생성됨")

전체 문서 처리 시작 (대상: 100건)...


 39%|███▉      | 39/100 [03:37<06:20,  6.23s/it]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



100%|██████████| 100/100 [09:38<00:00,  5.79s/it]


처리 완료: 총 98개의 Document 객체 생성됨


## 6. 문서 청킹 (Chunking) 및 Vector DB 구축
OpenAI 임베딩을 사용하여 벡터 DB를 구축합니다.

In [7]:
# ==========================================
# 1. 청킹 (Splitting)
# ==========================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""] # 한국어 우선순위
)

if documents:
    split_docs = text_splitter.split_documents(documents)
    print(f"청킹 완료: {len(documents)}개 문서 -> {len(split_docs)}개 청크 생성")
else:
    print("오류: 처리된 문서가 없습니다.")

# ==========================================
# 2. Vector DB 구축 (OpenAI + Chroma)
# ==========================================
# API Key 입력 (환경변수에 없으면 입력창 뜸)
# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key 입력: ")

print(f"Vector DB 구축 시작 (Embedding: text-embedding-3-small)...")
DB_PATH = "./chroma_db_bid_mate"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=DB_PATH,
    collection_name="bid_rfp_data"
)

print(f"DB 구축 완료! 저장 경로: {DB_PATH}")

청킹 완료: 98개 문서 -> 3529개 청크 생성
Vector DB 구축 시작 (Embedding: text-embedding-3-small)...
DB 구축 완료! 저장 경로: ./chroma_db_bid_mate


## 7. RAG 시스템 완성 (LLM 연결 및 테스트)
최종적으로 검색기(Retriever)와 GPT-4o-mini를 연결하여 질문에 답변합니다.

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

template = """
당신은 '입찰메이트'의 수석 컨설턴트입니다.
아래 [참고 문서]를 바탕으로 질문에 대해 핵심 정보(예산, 기간, 자격 등)를 포함하여 답변하세요.
문서에 없는 내용은 "정보가 없습니다"라고 답하고, 지어내지 마세요.

[참고 문서]
{context}

질문: {question}
답변:
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([f"<출처: {d.metadata['original_filename']}>\n{d.page_content}" for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 9. 최종 테스트
query = "학사정보시스템 고도화 사업의 예산과 주요 과업 내용을 요약해줘."
print(f"\n질문: {query}\n")
print("답변 생성 중...", end=" ")
response = rag_chain.invoke(query)

print("\n" + "="*60)
print(response)
print("="*60)


질문: 학사정보시스템 고도화 사업의 예산과 주요 과업 내용을 요약해줘.

답변 생성 중... 
학사정보시스템 고도화 사업의 예산은 130,000,000원(VAT 포함)입니다. 주요 과업 내용은 다음과 같습니다:

- 트랙기반 교육과정의 운영 및 관리 체계를 효과적으로 지원
- 교수자·학습자 중심의 교육환경 조성을 통한 대학 교육의 가치 구현
- 학사운영 시스템을 통해 대학 체제 개편에 대한 대응체계 확립
- 트랙제도 교과과정 개편 및 표준운영관리를 위한 시스템 구현
- 트랙제도 기반 교육과정과 현행 종합정보시스템 연계 가능하도록 고도화
- 확장성과 유연성을 고려한 환경 조성 및 유지보수가 용이하도록 해야 함
- 다양한 사용자(교수, 직원, 학생)의 정보 접근성 및 사용 편의성 고려
